# Environment Setup for PrecisionTrack Evaluation

## Introduction
This notebook is dedicated to evaluating the inference performance of PrecisionTrack in terms of both tracking quality and latency. Below are the steps to set up the necessary environment on Ubuntu 22.04.

## Installation

### Create Conda Environment
To begin, create a Conda environment named `precision_track` with the necessary packages. Run the following commands in your terminal:

```bash
conda create -n precision_track python==3.11.13
conda activate precision_track
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
pip install precision_track
```

In [1]:
import psutil
from mmengine import Config
import multiprocessing as mp
import os
import time

from precision_track import PipelinedTracker, Result, Visualizer
from precision_track.utils import VideoReader, infer_paths, find_path_in_dir
from precision_track.evaluation.utils.mot import evaluate_mot

mp.set_start_method("spawn", force=True)

In [2]:
cfg_path = "../configs/tasks/tracking.py"
video_path = "../assets/20mice.avi"
sink = "../assets/20mice_w_results.mp4"
gt_path = "../tests/work_dir/gt_20mice.csv"
cfg = Config.fromfile(cfg_path)

In [3]:
def launch_tracking(config, video):
    video = VideoReader(video)
    nb_cpu_cores = psutil.cpu_count(logical=False)
    if nb_cpu_cores >= 3 and config.pipelined:
        tracker = PipelinedTracker(
            detector=config.get("detector"),
            assigner=config.get("assigner"),
            validator=config.get("validator"),  # With Tailtags
            analyzer=None,
            outputs=config.get("outputs"),
            expected_resolution=video.resolution + (3,),
            batch_size=config.get("batch_size"),
            verbose=True,
        )
        t0 = time.perf_counter()
        tracker(video=video)
    delay_per_frame = (time.perf_counter() - t0) / len(video)
    print(f"Inference + saving the results took about {delay_per_frame:.3f}s per frame. Giving us a total performance of: {(1/delay_per_frame):.3f} FPS.")


launch_tracking(config=cfg, video=video_path)

06/12 12:10:14 - mmengine - INFO - Inference backend set to: TensorRT: 10.11.0.33 with FP16 precision on device: cuda.
06/12 12:10:14 - mmengine - INFO - Dynamically updating tracking thresholds to: {'low_thr': 0.05, 'conf_thr': 0.35, 'init_thr': 0.65}.


1471it [00:18, 80.76it/s] 


Saved output: /home/vincent/Documents/precision_track/work_dir/bboxes.csv
Saved output: /home/vincent/Documents/precision_track/work_dir/kpts.csv
Saved output: /home/vincent/Documents/precision_track/work_dir/velocities.csv
Saved output: /home/vincent/Documents/precision_track/work_dir/search_areas.csv
Inference + saving the results took about 0.014s per frame. Giving us a total performance of: 71.525 FPS.


In [17]:
metafile = cfg.get("metainfo", None)
gt_paths = infer_paths(gt_path)
result_paths = [cfg["outputs"][0]["path"]]

assert len(gt_paths) == len(result_paths)
gt_r_map = []
for i, gt in enumerate(gt_paths):
    assert os.path.exists(gt)
    gt_name = os.path.splitext(os.path.basename(gt))[0].replace("gt_", "")
    gt_idx = find_path_in_dir(gt_name, result_paths)
    gt_r_map.append((i, gt_idx))

print(f"Evaluating on {len(result_paths)} labelled videos...")
motas = []
idf1s = []
weigths = []
for i in gt_r_map:
    gt_idx, result_idx = i

    evaluation_results = evaluate_mot(
        result_paths[result_idx],
        gt_paths[gt_idx],
        metafile,
        save_path=None,
        verbose=True,
    )
    for cls_metrics in evaluation_results.values():
        motas.append(cls_metrics["mota"])
        idf1s.append(cls_metrics["idf1"])
        weigths.append(cls_metrics["num_detections"])
total_detections = sum(weigths)
weigths = [w / total_detections for w in weigths]
avg_mota = sum(m * w for m, w in zip(motas, weigths))
avg_idf1 = sum(i * w for i, w in zip(idf1s, weigths))
print(f"Weighted averages: MOTA={avg_mota:.3f}, IDF1={avg_idf1:.3f}.")

Evaluating on 1 labelled videos...
Progress: |██████████████████████████████████████████████████| 1470/1470

| Metric                  |     Score |
|-------------------------|-----------|
| MOTA on mouse           |     0.956 |
| IDF1 on mouse           |     0.943 |
| IDP on mouse            |     0.951 |
| IDR on mouse            |     0.936 |
| Precision on mouse      |     0.986 |
| Recall on mouse         |     0.970 |
| IDFP on mouse           |  1414.000 |
| IDFN on mouse           |  1881.000 |
| IDTP on mouse           | 27335.000 |
| Num Switches on mouse   |     8.000 |
| Num Detections on mouse | 28341.000 |
Weighted averages: MOTA=0.956, IDF1=0.943.


In [44]:
result = Result(outputs=cfg.get("outputs"))
result.read()
visualizer = Visualizer(**cfg.get("visualizer"))
visualizer(video_path, result, sink)
print(f"A qualitative assessment of the tracking performances has been saved at {sink}.")

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1471/1471, 19.0 task/s, elapsed: 77s, ETA:     0s
A qualitative assessment of the tracking performances has been saved at ../assets/20mice_w_results.mp4.
